## **Round 6**

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # displays all columns in dfs
#%matplotlib inline
import warnings
warnings.filterwarnings('ignore') # no more warnings

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
data = pd.read_csv('marketing_customer_analysis.csv')

In [4]:
data.shape

(9125, 24)

##### First of all an X-y split, to store 'total_claim_amount' separately from the rest of the data

In [5]:
y = data['total_claim_amount']
X = data.drop(['total_claim_amount'], axis=1)

In [6]:
X.isna().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
vehicle_class                    0
vehicle_size                     0
dtype: int64

##### Now separating numerical from categorical data

In [7]:
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = 'object')

In [8]:
X_cat

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,LA72316,California,No,Basic,Bachelor,2/10/11,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9121,PK87824,California,Yes,Extended,College,2/12/11,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9122,TD14365,California,No,Extended,Bachelor,2/6/11,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize
9123,UP19263,California,No,Extended,College,2/3/11,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


##### DROP: customer, effective_to_date. They would be a problem if I moved on to encoding

In [9]:
X_cat = X_cat.drop(['customer', 'effective_to_date'], axis=1)
X_cat

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9121,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9122,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize
9123,California,No,Extended,College,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


#### Moving on to encoding

In [10]:
# First of all, I need to check out the various categorical variables contained in each column

for column in X_cat.columns:
    print(column)
    print(X_cat[column].unique())

state
['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
response
['No' 'Yes']
coverage
['Basic' 'Extended' 'Premium']
education
['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
employmentstatus
['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
gender
['F' 'M']
location_code
['Suburban' 'Rural' 'Urban']
marital_status
['Married' 'Single' 'Divorced']
policy_type
['Corporate Auto' 'Personal Auto' 'Special Auto']
policy
['Corporate L3' 'Personal L3' 'Corporate L2' 'Personal L1' 'Special L2'
 'Corporate L1' 'Personal L2' 'Special L1' 'Special L3']
renew_offer_type
['Offer1' 'Offer3' 'Offer2' 'Offer4']
sales_channel
['Agent' 'Call Center' 'Web' 'Branch']
vehicle_class
['Two-Door Car' 'Four-Door Car' 'SUV' 'Luxury SUV' 'Sports Car'
 'Luxury Car']
vehicle_size
['Medsize' 'Small' 'Large']


In [11]:
# Now that that is known, I will start encoding them
# state: I will group the states as Northwestern = Washington, Oregon = nw and Southwestern = Arizona, California, Nevada = sw
# I will later encode these 2 values with onehot encoder

X_cat['state'] = X_cat['state'].apply(lambda x: str(x))
X_cat['state'] = (X_cat['state'].str.replace('Washington', 'nw')
                                    .replace('Oregon', 'nw')
                                    .replace('Arizona', 'sw')
                                    .replace('California', 'sw')
                                    .replace('Nevada','sw'))


In [12]:
# response only has 2 unique values. I will work on variables to be One-Hot encoded as a bundle
# coverage has 3 values --> onehot
# education: I will group the values as above. up to highschool = ah and highschool or below = hb. I will then use onehot encoder

X_cat['education'] = X_cat['education'].apply(lambda x: str(x))
X_cat['education'] = (X_cat['education'].str.replace('Bachelor', 'ah')
                                            .replace('College', 'ah')
                                            .replace('Master', 'ah')
                                            .replace('Doctor', 'ah')
                                            .replace('High School or Below','bh'))

In [13]:
# employmentstatus: I will use labels in this case as I can not create a simpler system of reference

X_cat['employmentstatus'] = X_cat['employmentstatus'].apply(lambda x: str(x))
X_cat['employmentstatus'] = (X_cat['employmentstatus'].str.replace('Employed', '1')
                                                        .replace('Unemployed', '2')
                                                        .replace('Medical Leave', '3')
                                                        .replace('Disabled', '4')
                                                        .replace('Retired', '5'))
X_cat['employmentstatus'] = X_cat['employmentstatus'].apply(lambda x: int(x))

In [14]:
# policy: I can decrease the number of columns to 3
# 1 = Corporate L1, Corporate L2, Corporate L3
# 2 = Personal L1, Personal L2, Personal L3
# 3 = Special L1, Special L2, Special L3
# then onehot encoding

X_cat['policy'] = X_cat['policy'].apply(lambda x: str(x))
X_cat['policy'] = (X_cat['policy'].str.replace('Corporate L1', 'Corporate')
                                        .replace('Corporate L2', 'Corporate')
                                        .replace('Corporate L3', 'Corporate')
                                        .replace('Personal L1', 'Personal')
                                        .replace('Personal L2', 'Personal')
                                        .replace('Personal L3', 'Personal')
                                        .replace('Special L1', 'Special')
                                        .replace('Special L2', 'Special')
                                        .replace('Special L3', 'Special'))

In [15]:
# renew offer type: I will work with labels again

X_cat['renew_offer_type'] = X_cat['renew_offer_type'].apply(lambda x: str(x))
X_cat['renew_offer_type'] = (X_cat['renew_offer_type'].str.replace('Offer1', '1')
                                                        .replace('Offer2', '2')
                                                        .replace('Offer3', '3')
                                                        .replace('Offer4', '4'))
X_cat['renew_offer_type'] = X_cat['renew_offer_type'].apply(lambda x: int(x))

In [16]:
#sales channel: again labels

X_cat['sales_channel'] = X_cat['sales_channel'].apply(lambda x: str(x))
X_cat['sales_channel'] = (X_cat['sales_channel'].str.replace('Agent', '1')
                                                        .replace('Call Center', '2')
                                                        .replace('Web', '3')
                                                        .replace('Branch', '4'))
X_cat['sales_channel'] = X_cat['sales_channel'].apply(lambda x: int(x))

In [17]:
#vehicle class: labels

X_cat['vehicle_class'] = X_cat['vehicle_class'].apply(lambda x: str(x))                                                        
X_cat['vehicle_class'] = (X_cat['vehicle_class'].str.replace('Two-Door Car', '1')                                                        
                                                        .replace('Four-Door Car', '2')
                                                        .replace('SUV', '3')
                                                        .replace('Luxury SUV', '4')
                                                        .replace('Luxury Car', '4') #decrease number of columns by one
                                                        .replace('Sports Car', '5'))
X_cat['vehicle_class'] = X_cat['vehicle_class'].apply(lambda x: int(x))

In [18]:
# Now I will create a new dataframe containing only the label encoded columns

Xnothot = X_cat[['employmentstatus', 'renew_offer_type', 'sales_channel', 'vehicle_class']]
Xnothot = Xnothot.reset_index(drop=True)

In [19]:
# And then another one, containing the rest that will be onehot encoded

Xhot = X_cat[['state', 'response', 'coverage', 'education', 'gender', 'location_code', 'marital_status', 'policy_type', 'policy', 'vehicle_size']]
encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(Xhot)

#state

{'nw': [1,0],
'sw': [0,0]}

# response only has 2 unique values and will be thus easy to encode

{'Yes': [1,0],
'No': [0,0]}

# coverage has 3 values, thus I will follow the same method as Xisca

{'Basic': [1,0],
'Extended': [0,0],
'Premium': [0,1]}

#education

{'ah': [1,0],
'hb': [0,0]}

#gender tick

{'M': [1,0],
'F': [0,0]}

#location: like xisca

{'Suburban': [1,0],
'Rural': [0,0],
'Urban': [0,1]}

#marital: like xisca

{'Married': [1,0],
'Single': [0,0],
'Divorced': [0,1]}

#policy_type: same

{'Corporate Auto': [1,0],
'Personal Auto': [0,0],
'Special Auto': [0,1]}

#policy

{'Corporate': [1,0],
'Personal': [0,0],
'Special': [0,1]}

#vehicle size: like xisca

{'Medsize': [1,0],
'Small': [0,0],
'Large': [0,1]}

encoded = encoder.transform(Xhot).toarray() 
cat_enc1 = pd.DataFrame(encoded)
#encoder.categories_
#cat_encoded.columns = encoder.categories_[0][1:] A more complicated way of assigning the column names
cat_enc1.columns = ['state_nw', 'response_yes','cov_basic', 'cov_premium', 'edu_ab_high', 'gender_m', 'suburban', 'urban', 'married', 'divorced', 'corporate_auto', 'special_auto', 'policy_corp', 'policy_special', 'veh_size_medsize', 'veh_size_large']
cat_enc1

,state_nw,response_yes,cov_basic,cov_premium,edu_ab_high,gender_m,suburban,urban,married,divorced,corporate_auto,special_auto,policy_corp,policy_special,veh_size_medsize,veh_size_large
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9121,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9122,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9123,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [20]:
# The columns corporate_auto and special_auto, initially part of policy_type ended up having exactly the same output as policy_corp and policy_special respectively and are thus redundant

cat_enc1 = cat_enc1.drop(['corporate_auto', 'special_auto'], axis=1)

In [21]:
# To create a new dataframe with all of my encoded categoricals

cat_enc2 = pd.concat([Xnothot, cat_enc1], axis=1)
cat_enc2

,employmentstatus,renew_offer_type,sales_channel,vehicle_class,state_nw,response_yes,cov_basic,cov_premium,edu_ab_high,gender_m,suburban,urban,married,divorced,policy_corp,policy_special,veh_size_medsize,veh_size_large
0,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,2,3,1,2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1,1,1,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,2,1,2,3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,1,1,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,1,2,3,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
9121,1,1,4,2,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9122,2,1,4,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9123,1,3,4,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [22]:
X_num = X_num.reset_index(drop=True)
X_num

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,2763.519279,56274,69,32,5,0,1
1,6979.535903,0,94,13,42,0,8
2,12887.431650,48767,108,18,38,0,2
3,7645.861827,0,106,18,65,0,7
4,2813.692575,43836,73,12,44,0,1
...,...,...,...,...,...,...,...
9120,23405.987980,71941,73,18,89,0,2
9121,3096.511217,21604,79,14,28,0,1
9122,8163.890428,0,85,9,37,3,2
9123,7524.442436,21941,96,34,3,0,3


*As seen in the previous lab (5), the number of open complaints was not a variable that we could manipulate in a satisfactory way with scaling and transforming. I will group its data points as 1 in case of any open complaints or 0 in case of none*

In [23]:
X_num['number_of_open_complaints'].unique()

array([0, 2, 1, 3, 5, 4], dtype=int64)

In [24]:
X_num['number_of_open_complaints'] = X_num['number_of_open_complaints'].apply(lambda x: str(x))
X_num['number_of_open_complaints'] = (X_num['number_of_open_complaints'].str.replace('2', '1')
                                                        .replace('3', '1')
                                                        .replace('4', '1')
                                                        .replace('5', '1'))
X_num['number_of_open_complaints'] = X_num['number_of_open_complaints'].apply(lambda x: int(x))

#### And now I will create a dataframe containing all the encoded categoricals and numericals

In [26]:
X = pd.concat([X_num, cat_enc2], axis=1)
X

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,employmentstatus,renew_offer_type,sales_channel,vehicle_class,state_nw,response_yes,cov_basic,cov_premium,edu_ab_high,gender_m,suburban,urban,married,divorced,policy_corp,policy_special,veh_size_medsize,veh_size_large
0,2763.519279,56274,69,32,5,0,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,6979.535903,0,94,13,42,0,8,2,3,1,2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,12887.431650,48767,108,18,38,0,2,1,1,1,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,7645.861827,0,106,18,65,0,7,2,1,2,3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,2813.692575,43836,73,12,44,0,1,1,1,1,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,23405.987980,71941,73,18,89,0,2,1,2,3,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
9121,3096.511217,21604,79,14,28,0,1,1,1,4,2,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9122,8163.890428,0,85,9,37,1,2,2,1,4,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9123,7524.442436,21941,96,34,3,0,3,1,3,4,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


### **Linear Regression**
#### -Train-test split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=56) #70% rows meant to be used for training, 30% for testing

In [28]:
len(X_test)

2738

In [29]:
len(X)

9125

In [30]:
len(X_train)

6387

In [31]:
X_train.head()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,employmentstatus,renew_offer_type,sales_channel,vehicle_class,state_nw,response_yes,cov_basic,cov_premium,edu_ab_high,gender_m,suburban,urban,married,divorced,policy_corp,policy_special,veh_size_medsize,veh_size_large
2090,5016.827946,20868,63,19,13,0,8,1,3,1,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2544,2700.985899,79292,67,18,58,0,1,1,2,4,2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8895,6480.099944,18820,95,3,51,1,8,3,1,4,1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3120,3130.678430,42056,78,21,2,0,1,1,1,1,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4832,4272.805040,0,114,0,75,0,1,2,1,2,2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


#### -Application of linear regression

In [32]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [33]:
predictions  = model.predict(X_test)
predictions

array([237.19684666, 566.68950053, 768.77174712, ..., 515.59727393,
       382.9265856 , 693.97926772])

## **Model Evaluation**

In [35]:
r2 = r2_score(y_test, predictions)
RMSE = mean_squared_error(y_test, predictions, squared=False)
MSE = mean_squared_error(y_test, predictions)
MAE = mean_absolute_error(y_test, predictions)

print("r2 = ", round(r2, 2))
print("RMSE = ", round(RMSE, 2))
print("MSE = ", round(MSE, 2))
print("MAE = ", round(MAE))


r2 =  0.76
RMSE =  140.42
MSE =  19718.55
MAE =  96


In [36]:
rounded_td = round(data['total_claim_amount'].describe(), 2)
rounded_td

count    9125.00
mean      433.13
std       288.65
min         0.10
25%       272.02
50%       383.51
75%       547.20
max      2893.24
Name: total_claim_amount, dtype: float64

##### *It looks like our predictions are not good enough, since the RMSE is almost 1/3 of the average total claim amount and the MAE close to 1/4 of the same value*

In [39]:
data_new = pd.concat([X, y], axis=1)
data_new.to_csv('marketing_customer_analysis.csv', sep=',', index=False)